https://github.com/mlflow/mlflow

MLflow - это инструмент для управления жизненным циклом машинного обучения: отслеживание экспериментов,  
управление и деплой моделей и проектов. В этом руководстве мы посмотрим, как организовать эксперименты и запуски,  
оптимизировать гиперпараметры с помощью optuna, сравнивать модели и выбирать лучшие параметры.   
Также рассмотрим логирование моделей, использование их в разных форматах, упаковку проекта в MLproject и установку удаленного Tracking Server MLflow.

In [ ]:
!pip uninstall mlflow

Можно просто установить mlflow и испольовать

Можно установить conda и создать новоет окружение:

In [ ]:
name: mlflow-example
channels:
  - conda-forge
  - defaults
dependencies:
  - python=3.11.4
  - pip=24.0
  - mlflow=2.14.2
  - xgboost=2.0.3
  - jupyter=1.0.0
  - loguru=0.7.2
  - shap=0.45.1
  - pandas=2.2.2
  - scikit-learn=1.5.1
  - scipy=1.14.0
  - numpy=1.26.4
  - jupytext=1.16.3
  - psutil=6.0.0
  - boto3=1.34.148
  - psycopg2-binary=2.9.9
  - pip:
    - optuna==3.6.1
    - mlserver==1.3.5
    - mlserver-mlflow==1.3.5
    - mlserver-xgboost==1.3.5

Можно установить вручную или сохранить этот файл локально и вызвать:  

conda env create -f conda.yaml  
Теперь можно использовать это окружение в IDE или в терминале:  

conda activate mlflow-example  

Выполните в терминале mlflow ui, чтобы запустить MLflow на localhost:5000.

Mlflow будет запущен в локальном режиме, по умолчанию он создаст папку mlruns для хранения артефактов и метаинформации.

In [ ]:
!mlflow ui

Далее просто передаем в метод `mlflow.log_metric` то, что хотим трекать 

In [ ]:
import mlflow

def log_metrics(rmse_train, 
                mae_train, 
                r2_train, 
                rmse_valid, 
                mae_valid, 
                r2_valid):
    mlflow.log_metric('rmse_train', rmse_train)
    mlflow.log_metric('mae_train', mae_train)
    mlflow.log_metric('r2_train', r2_train)
    
    mlflow.log_metric('rmse_valid', rmse_valid)
    mlflow.log_metric('mae_valid', mae_valid)
    mlflow.log_metric('r2_valid', r2_valid)

In [ ]:
def log_parameters(model, only_model_name=True, **params):
    name = model.__class__.__name__
    if only_model_name:
        mlflow.log_param('model_name', name)
    else:
        mlflow.sklearn.log_model(model, name)
    mlflow.log_params(params)

Теперь указываем, что мы будем проводить эксперимент такой-то  
По сути создасться папка и в нее будет все скаладываться

In [ ]:
mlflow.set_experiment('Boston')

Используем примерно так

In [ ]:
X_train, y_train, X_valid, y_valid = fixed_train_test_split(df)

clf = LinearRegression()
clf.fit(X_train, y_train)

y_pred_t = clf.predict(X_train)
y_pred = clf.predict(X_valid)

rt, mt, r2t = calc_metrics(y_train, y_pred_t)
r, m, r2 = calc_metrics(y_valid, y_pred)

# создаем новую строчку в таблице и называем ее 'Baseline'
with mlflow.start_run(run_name='Baseline'):
    log_metrics(rt, mt, r2t, r, m, r2)
    log_parameters(clf)

Теперь нам нужно из папки с проектом запустить в командной строке  
`mlflow ui`  
У нас в папке проекта будет папка `mlflow` с данными, которые мы трекали,  
поэтому нужно запускать команду `mlflow ui` из папки проекта